In [1]:
!wget https://huggingface.co/datasets/puhsu/tabular-benchmarks/resolve/main/data.tar -O tabular-dl-tabr.tar.gz
!tar -xvf tabular-dl-tabr.tar.gz

--2024-10-15 14:02:37--  https://huggingface.co/datasets/puhsu/tabular-benchmarks/resolve/main/data.tar
Resolving huggingface.co (huggingface.co)... 18.239.50.103, 18.239.50.80, 18.239.50.16, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.103|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/1b/25/1b25d6e2556c827abfaf6ba9da6021338691cafc39fe9d77986955ae0a69243d/0d74e4b96febb48fbe4dd2d851f8638b5738354f82c3183f92adbd2abf2f256b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27data.tar%3B+filename%3D%22data.tar%22%3B&response-content-type=application%2Fx-tar&Expires=1729260158&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyOTI2MDE1OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8xYi8yNS8xYjI1ZDZlMjU1NmM4MjdhYmZhZjZiYTlkYTYwMjEzMzg2OTFjYWZjMzlmZTlkNzc5ODY5NTVhZTBhNjkyNDNkLzBkNzRlNGI5NmZlYmI0OGZiZTRkZDJkODUxZjg2MzhiNTczODM1NGY4MmMzMTgzZjkyYWRiZDJhYmYyZjI1Nm

In [2]:
!pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 3.3 MB/s eta 0:00:00


In [3]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.6 MB/s eta 0:00:00


In [4]:
!pip install psutil

In [5]:
import pandas as pd
import numpy as np
import optuna
import torch.optim
import psutil

from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.preprocessing import QuantileTransformer

In [6]:
tnr = TabNetRegressor()

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [7]:
X_train = np.load('/content/data/regression-num-medium-0-superconduct/X_num_train.npy')
y_train = np.load('/content/data/regression-num-medium-0-superconduct/Y_train.npy').reshape(-1, 1)

X_valid = np.load('/content/data/regression-num-medium-0-superconduct/X_num_val.npy')
y_valid = np.load('/content/data/regression-num-medium-0-superconduct/Y_val.npy').reshape(-1, 1)

X_test = np.load('/content/data/regression-num-medium-0-superconduct/X_num_test.npy')
y_test = np.load('/content/data/regression-num-medium-0-superconduct/Y_test.npy').reshape(-1, 1)

In [8]:
normalizer = QuantileTransformer(
            output_distribution='normal',
            n_quantiles=max(min(X_train.shape[0] // 30, 1000), 10),
            subsample=1_000_000_000,
            )
normalizer.fit_transform(X_train)

array([[-0.34656826, -0.38103637, -0.525267  , ...,  0.079359  ,
        -0.8298449 , -0.30274472],
       [-0.41147688, -0.7503491 , -0.45711228, ..., -1.0138142 ,
        -0.8298449 , -1.1835144 ],
       [ 0.17070231, -0.5639779 ,  0.31863937, ..., -0.5226079 ,
         0.        , -0.30307585],
       ...,
       [ 0.41134915, -0.9937043 ,  0.5597231 , ..., -0.3287044 ,
         1.1430639 ,  0.6274264 ],
       [ 2.1042554 ,  1.6814585 ,  2.115818  , ...,  1.9873234 ,
        -5.1993375 , -5.1993375 ],
       [-2.0376847 , -1.8410197 , -1.9418334 , ...,  0.4866506 ,
         1.3572593 ,  1.7786256 ]], dtype=float32)

In [9]:
#Baseline
tnr.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['mae', 'rmse'],
)

epoch 0  | loss: 2276.83499| train_mae: 25.3787899017334| train_rmse: 36.09175109863281| valid_mae: 25.528860092163086| valid_rmse: 36.10728073120117|  0:00:02s
epoch 1  | loss: 1884.15051| train_mae: 35.32780075073242| train_rmse: 47.541439056396484| valid_mae: 35.25843811035156| valid_rmse: 47.465938568115234|  0:00:03s
epoch 2  | loss: 1340.63341| train_mae: 23.184179306030273| train_rmse: 34.705108642578125| valid_mae: 23.43716049194336| valid_rmse: 34.443660736083984|  0:00:04s
epoch 3  | loss: 784.43372| train_mae: 34.521610260009766| train_rmse: 64.39351654052734| valid_mae: 35.17496109008789| valid_rmse: 65.18331909179688|  0:00:05s
epoch 4  | loss: 405.41551| train_mae: 37.86082077026367| train_rmse: 55.63283920288086| valid_mae: 39.23923873901367| valid_rmse: 57.47438049316406|  0:00:05s
epoch 5  | loss: 295.96734| train_mae: 64.38655853271484| train_rmse: 86.10163116455078| valid_mae: 64.98332977294922| valid_rmse: 86.55345916748047|  0:00:06s
epoch 6  | loss: 284.73232| tra

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [10]:
def objective(trial):
    params = {
        'n_d': trial.suggest_int('n_d', 8, 64),
        'n_steps': trial.suggest_int('n_steps', 3, 10),
        'gamma': trial.suggest_float('gamma', 1.0, 2.0),
        'n_independent': trial.suggest_int('n_independent', 1, 5),
        'n_shared': trial.suggest_int('n_shared', 1, 5),
        'momentum': trial.suggest_float('momentum', 0.01, 0.4),
    }

    model = TabNetRegressor(**params, n_a=params['n_d'])
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], patience=10)

    # Evaluate model performance
    score = model.best_cost  # or any other metric

    return score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

best_params = study.best_params

[I 2024-10-15 14:04:10,382] A new study created in memory with name: no-name-41e80d4f-d928-4328-8447-90069d901cc8
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1898.05438| val_0_mse: 55153.2890625|  0:00:01s
epoch 1  | loss: 1106.23966| val_0_mse: 21754.7578125|  0:00:02s
epoch 2  | loss: 728.78733| val_0_mse: 5637.7783203125|  0:00:04s
epoch 3  | loss: 569.86243| val_0_mse: 2244.233642578125|  0:00:05s
epoch 4  | loss: 533.65029| val_0_mse: 3143.89306640625|  0:00:06s
epoch 5  | loss: 545.53985| val_0_mse: 2322.34033203125|  0:00:08s
epoch 6  | loss: 495.1388| val_0_mse: 1252.04296875|  0:00:09s
epoch 7  | loss: 464.97385| val_0_mse: 1339.28955078125|  0:00:10s
epoch 8  | loss: 462.28496| val_0_mse: 1530.982666015625|  0:00:12s
epoch 9  | loss: 457.66401| val_0_mse: 1859.3060302734375|  0:00:13s
epoch 10 | loss: 440.1539| val_0_mse: 1479.030029296875|  0:00:14s
epoch 11 | loss: 437.69959| val_0_mse: 1282.3148193359375|  0:00:16s
epoch 12 | loss: 424.30404| val_0_mse: 1072.7437744140625|  0:00:17s
epoch 13 | loss: 471.23288| val_0_mse: 993.0364990234375|  0:00:18s
epoch 14 | loss: 483.44754| val_0_mse: 934.40576171875|  0:00:

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-10-15 14:04:53,946] Trial 0 finished with value: 527.45654296875 and parameters: {'n_d': 31, 'n_steps': 7, 'gamma': 1.35516410637792, 'n_independent': 4, 'n_shared': 4, 'momentum': 0.2768196243893071}. Best is trial 0 with value: 527.45654296875.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1616.29003| val_0_mse: 32790.71484375|  0:00:00s
epoch 1  | loss: 688.35637| val_0_mse: 4299.87646484375|  0:00:01s
epoch 2  | loss: 519.99551| val_0_mse: 1083.519775390625|  0:00:02s
epoch 3  | loss: 459.74799| val_0_mse: 1615.077392578125|  0:00:03s
epoch 4  | loss: 446.08189| val_0_mse: 823.8484497070312|  0:00:04s
epoch 5  | loss: 401.82027| val_0_mse: 1006.6163330078125|  0:00:05s
epoch 6  | loss: 377.93265| val_0_mse: 1507.535400390625|  0:00:06s
epoch 7  | loss: 356.94058| val_0_mse: 1246.75634765625|  0:00:07s
epoch 8  | loss: 364.87153| val_0_mse: 720.806396484375|  0:00:08s
epoch 9  | loss: 357.14077| val_0_mse: 843.5452270507812|  0:00:09s
epoch 10 | loss: 357.24363| val_0_mse: 608.2981567382812|  0:00:10s
epoch 11 | loss: 339.63211| val_0_mse: 673.8869018554688|  0:00:11s
epoch 12 | loss: 328.05112| val_0_mse: 651.09716796875|  0:00:12s
epoch 13 | loss: 315.18334| val_0_mse: 529.9465942382812|  0:00:13s
epoch 14 | loss: 317.63357| val_0_mse: 358.65515136718

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-10-15 14:06:09,977] Trial 1 finished with value: 193.3863983154297 and parameters: {'n_d': 60, 'n_steps': 5, 'gamma': 1.5158429596467773, 'n_independent': 5, 'n_shared': 2, 'momentum': 0.20855953208246553}. Best is trial 1 with value: 193.3863983154297.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


...

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-10-15 15:35:55,976] Trial 97 finished with value: 153.22933959960938 and parameters: {'n_d': 35, 'n_steps': 4, 'gamma': 1.6693459581717853, 'n_independent': 3, 'n_shared': 5, 'momentum': 0.13219105241800796}. Best is trial 33 with value: 135.8055877685547.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1489.28921| val_0_mse: 108163.5703125|  0:00:01s
epoch 1  | loss: 731.62916| val_0_mse: 8711.30859375|  0:00:02s
epoch 2  | loss: 588.30362| val_0_mse: 2113.103515625|  0:00:03s
epoch 3  | loss: 537.63578| val_0_mse: 6676.744140625|  0:00:04s
epoch 4  | loss: 520.23142| val_0_mse: 1248.8671875|  0:00:05s
epoch 5  | loss: 452.03021| val_0_mse: 1413.9654541015625|  0:00:07s
epoch 6  | loss: 443.46328| val_0_mse: 2315.471435546875|  0:00:08s
epoch 7  | loss: 452.6895| val_0_mse: 705.3342895507812|  0:00:09s
epoch 8  | loss: 498.82333| val_0_mse: 2439.140625|  0:00:10s
epoch 9  | loss: 449.07606| val_0_mse: 1374.469482421875|  0:00:11s
epoch 10 | loss: 433.79872| val_0_mse: 2202.276123046875|  0:00:12s
epoch 11 | loss: 429.75564| val_0_mse: 1080.1796875|  0:00:13s
epoch 12 | loss: 391.3453| val_0_mse: 1038.591064453125|  0:00:14s
epoch 13 | loss: 393.93135| val_0_mse: 1012.6807250976562|  0:00:16s
epoch 14 | loss: 366.07558| val_0_mse: 1156.622802734375|  0:00:17s
epoch 15

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-10-15 15:37:50,131] Trial 98 finished with value: 164.82093811035156 and parameters: {'n_d': 64, 'n_steps': 7, 'gamma': 1.4439139415056461, 'n_independent': 4, 'n_shared': 2, 'momentum': 0.2906631523981622}. Best is trial 33 with value: 135.8055877685547.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1585.84587| val_0_mse: 34502.75390625|  0:00:00s
epoch 1  | loss: 460.3007| val_0_mse: 11995.7734375|  0:00:01s
epoch 2  | loss: 373.14782| val_0_mse: 7181.12060546875|  0:00:01s
epoch 3  | loss: 294.2411| val_0_mse: 1925.7633056640625|  0:00:02s
epoch 4  | loss: 300.53768| val_0_mse: 3048.811767578125|  0:00:03s
epoch 5  | loss: 283.88398| val_0_mse: 1949.3536376953125|  0:00:03s
epoch 6  | loss: 272.06189| val_0_mse: 2351.68408203125|  0:00:04s
epoch 7  | loss: 267.90889| val_0_mse: 3910.25048828125|  0:00:05s
epoch 8  | loss: 261.89424| val_0_mse: 1557.8394775390625|  0:00:05s
epoch 9  | loss: 244.94405| val_0_mse: 1480.325439453125|  0:00:06s
epoch 10 | loss: 252.45689| val_0_mse: 2222.34765625|  0:00:07s
epoch 11 | loss: 239.12446| val_0_mse: 2450.130859375|  0:00:07s
epoch 12 | loss: 230.81992| val_0_mse: 2278.248046875|  0:00:08s
epoch 13 | loss: 227.07229| val_0_mse: 2699.365966796875|  0:00:08s
epoch 14 | loss: 241.43877| val_0_mse: 2133.947998046875|  0:00:09

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-10-15 15:38:26,243] Trial 99 finished with value: 166.77169799804688 and parameters: {'n_d': 58, 'n_steps': 3, 'gamma': 1.4935322964190052, 'n_independent': 2, 'n_shared': 4, 'momentum': 0.20474979051373327}. Best is trial 33 with value: 135.8055877685547.


In [11]:
best_params

{'n_d': 57,
 'n_steps': 3,
 'gamma': 1.4326005028022137,
 'n_independent': 3,
 'n_shared': 3,
 'momentum': 0.10390725153746129}

In [12]:
from pytorch_tabnet.callbacks import Callback

def print_memory_usage():
    process = psutil.Process()
    mem_info = process.memory_info()
    print(f"Memory Usage: {mem_info.rss / (1024 ** 2):.2f} MB")

class MemoryUsageCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print_memory_usage()

In [13]:
# Train final model using best hyperparameters
best_model = TabNetRegressor(**best_params,
                             n_a=best_params['n_d'],
                             optimizer_fn=torch.optim.Adam,
                             optimizer_params=dict(lr=2e-2)
                             )
best_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['mae', 'rmse'],
    callbacks=[MemoryUsageCallback()]
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1635.88506| train_mae: 40.05009841918945| train_rmse: 63.12295913696289| valid_mae: 37.89181900024414| valid_rmse: 59.29082107543945|  0:00:00s
Memory Usage: 1223.48 MB
epoch 1  | loss: 451.87158| train_mae: 35.08042907714844| train_rmse: 45.933998107910156| valid_mae: 35.7258186340332| valid_rmse: 47.01129913330078|  0:00:01s
Memory Usage: 1223.48 MB
epoch 2  | loss: 344.96931| train_mae: 58.52035903930664| train_rmse: 77.70072174072266| valid_mae: 60.758018493652344| valid_rmse: 79.64662170410156|  0:00:02s
Memory Usage: 1223.48 MB
epoch 3  | loss: 299.25001| train_mae: 22.249000549316406| train_rmse: 31.95471954345703| valid_mae: 22.811370849609375| valid_rmse: 32.34431076049805|  0:00:03s
Memory Usage: 1223.48 MB
epoch 4  | loss: 293.87474| train_mae: 43.87417984008789| train_rmse: 64.5948486328125| valid_mae: 46.00606155395508| valid_rmse: 66.53350067138672|  0:00:04s
Memory Usage: 1223.48 MB
epoch 5  | loss: 296.14812| train_mae: 19.75589942932129| train_rmse: 29

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, classification_report

preds = best_model.predict(X_test)
y_true = y_test

mae_test = mean_absolute_error(y_pred=preds, y_true=y_true)
mse = mean_squared_error(y_pred=preds, y_true=y_true)
rmse_test = np.sqrt(mean_squared_error(y_true, y_pred=preds))
r2_test = r2_score(y_true=y_true, y_pred=preds)

print("Best Valid RMSE:", best_model.best_cost)
print("Test MAE:", mae_test)
print("Test RMSE:", rmse_test)
print("R-squared:", r2_test)

Best Valid RMSE: 12.825269
Test MAE: 8.69767
Test RMSE: 13.161221
R-squared: 0.8514436483383179


In [ ]:
base_preds = tnr.predict(X_test)

base_mae_test = mean_absolute_error(y_pred=base_preds, y_true=y_true)
base_mse = mean_squared_error(y_pred=base_preds, y_true=y_true)
base_rmse_test = np.sqrt(mean_squared_error(y_true, y_pred=base_preds))

print("Best Valid Score:", tnr.best_cost)
print("Test MAE:", base_mae_test)
print("Test RMSE:", base_rmse_test)

Best Valid Score: 34.443657
Test MAE: 23.186302
Test RMSE: 34.57977
